In [6]:
import json
from PIL import Image, ImageDraw 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import pickle
import numpy as np
import shutil

/tmp/ipykernel_448036/2704337986.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Config Directories

In [7]:
#BDD Annotation
vgRoot = 'VisualGenome/'

vgAttr = vgRoot+'Annotation/attributes.json'
vgObj = vgRoot+'Annotation/objects.json'
vgSG = vgRoot+'Annotation/scene_graphs.json'

vgExAttr = 'ExtractAttribute'
vgExRel = 'ExtractRelation'
vgGT = 'ExtractGT'
vgImg = 'VG_100K'

In [8]:
fVG = open(vgSG)
dataSG = json.load(fVG)
len(dataSG)

108077

In [132]:
objects_list_partial = [
    "railcar", "trafficlight", "street", "city employee", "passenger window", "road indicator", "cross walk sign",
    "traffic post", "street intersection", "bus that is white", "traffic cones", "street name", "traffic camera",
    "street lights", "traffic meter", "stop sign", "traffic light", "roads", "stop lights", "car door",
    "pedestrian sign", "bike racks", "construction sign", "pedestrian crossing", "railroad crossing", "railway track",
    "police car", "traffic signals", "bus", "road signs", "crosswalk", "bike", "pedestrian walk", "train tracks",
    "rail line", "street camera", "railng", "traffic indicators", "crosswalk signal", "street divider", "railtrack",
    "road barrier", "street light", "lane", "traffic line", "crosswalks", "handicap sign", "traffic lilght",
    "railroad track", "street post", "street side", "sidewalk grate", "street marker", "bus sign", "cross walk light",
    "traffic post", "road sign", "no right turn", "street lamp is green", "street pavement", "no stopping anytime",
    "speed limit", "metro sign", "rail crossing", "bus platform", "pedestrian light", "traffic cones", "railway",
    "convertible", "electric sign", "emergency vehicle", "pedestrian", "railroad track", "sidewalk", "street name sign",
    "train signal", "bus depot", "construction vehicle", "railroad track", "train station", "pedstrian", "railroad",
    "stop sign", "traffic meter", "railtrack", "street camera", "train platform", "crosswalk", "lane", "traffic light",
    "traffic meter", "stop lights", "train tracks", "bike path", "construction crane", "metro line", "parking meter",
    "street lights", "street name", "traffic camera", "bus stop", "car train", "pedestrian crossing", "rail crossing",
    "rail line", "street divider", "traffic light", "crosswalk signal", "railway track", "street light", "traffic sign",
    "bike lane", "construction sign", "pedestrian walk", "railroad crossing", "street sign", "traffic cone", "bike rack",
    "bus lane", "crosswalk light", "railroad sign", "roadwork sign", "stoplight", "traffic barrier", "pedestrian lane",
    "railway crossing", "speed bump", "street crossing", "traffic island", "bicycle stand", "car parking", "no parking sign",
    "parking garage", "pedestrian bridge", "road block", "sidewalk curb", "traffic circle", "bike trail", "crossing guard",
    "parking lot", "road sign", "street lamp", "taxicab", "traffic light", "bike lane", "bus stop", "parking space",
    "pedestrian path", "road barrier", "speed limit sign", "traffic signal", "zebra crossing", "carpool lane", "footpath",
    "motorcycle parking", "parking sign", "roadway", "traffic cone", "bicycle path", "crosswalk", "parking meter",
    "pedestrian zone", "roundabout", "street light", "traffic barrier", "bus zone", "cycle lane", "handicap parking",
    "no entry sign", "parking bay", "road marking", "traffic sign", "walkway", "bicycle lane", "car park", "driveway",
    "motorway", "park and ride", "pedestrian signal", "rail crossing", "street", "traffic cone", "vehicle lane",
    "zebra crossing", "alleyway", "bypass", "cul-de-sac", "footbridge", "junction", "pedestrian overpass", "roadside",
    "stop sign", "traffic island", "bicycle crossing"
]

In [225]:
sysnObj = open(vgRoot + 'Annotation/object_synsets.json')
objects_sysn = json.load(sysnObj)
len(objects_sysn)

40154

In [60]:
sysnRel = open(vgRoot + 'Annotation/relationship_synsets.json')
rel_sysn = json.load(sysnRel)
len(rel_sysn)

13536

In [61]:
sysnAttr = open(vgRoot + 'Annotation/attribute_synsets.json')
attr_sysn = json.load(sysnAttr)
len(attr_sysn)

18720

In [226]:
lsSysn = []
for item in objects_sysn.keys():
    lsSysn.append(objects_sysn[item])
    # break

print(len(lsSysn))
lsSysn = list(set(lsSysn))
print(len(lsSysn))

# open file in write mode
with open(vgRoot + 'Annotation/' +'sysn_obj.txt', 'w') as fp:
    for item in lsSysn:
        # write each item on a new line
        fp.write("%s\n" % item.strip())
    print('Done')

40154
5772
Done


In [55]:
# Using readlines()
objectNormTraffic = []
file1 = open(vgRoot+'Annotation/objectTraffic.txt', 'r')
Lines = file1.readlines()

count = 0
# Strips the newline character
for line in Lines:
    count += 1
    objectNormTraffic.append(line.strip().lower().replace('\n',''))

print(len(objectNormTraffic))
objectNormTraffic = objectNormTraffic[:]
len(objectNormTraffic)

objectNorm  = [ 'airplane', 'animal', 'arm', 'bag', 'banana', 'basket', 'beach', 'bear', 'bed', 'bench', 'bike',
                'bird', 'board', 'boat', 'book', 'boot', 'bottle', 'bowl', 'box', 'boy', 'branch', 'building',
                'bus', 'cabinet', 'cap', 'car', 'cat', 'chair', 'child', 'clock', 'coat', 'counter', 'cow', 'cup',
                'curtain', 'desk', 'dog', 'door', 'drawer', 'ear', 'elephant', 'engine', 'eye', 'face', 'fence',
                'finger', 'flag', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'guy',
                'hair', 'hand', 'handle', 'hat', 'head', 'helmet', 'hill', 'horse', 'house', 'jacket', 'jean',
                'kid', 'kite', 'lady', 'lamp', 'laptop', 'leaf', 'leg', 'letter', 'light', 'logo', 'man', 'men',
                'motorcycle', 'mountain', 'mouth', 'neck', 'nose', 'number', 'orange', 'pant', 'paper', 'paw',
                'people', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'plate', 'player', 'pole', 'post',
                'pot', 'racket', 'railing', 'rock', 'roof', 'room', 'screen', 'seat', 'sheep', 'shelf', 'shirt',
                'shoe', 'short', 'sidewalk', 'sign', 'sink', 'skateboard', 'ski', 'skier', 'sneaker', 'snow',
                'sock', 'stand', 'street', 'surfboard', 'table', 'tail', 'tie', 'tile', 'tire', 'toilet', 'towel',
                'tower', 'track', 'train', 'tree', 'truck', 'trunk', 'umbrella', 'vase', 'vegetable', 'vehicle',
                'wave', 'wheel', 'window', 'windshield', 'wing', 'wire', 'woman', 'zebra']

len(objectNorm)


170


150

In [10]:
objectNew = []
error = []
for item in dataSG:
    for x in item['objects']:
        try:
            if(x['synsets'][0].split('.')[0] in objectNorm):
                objectNew.append(item['image_id'])
                break
        except:
            error.append(item['image_id'])
    #break
print(len(objectNew))
objectNew = list(set(objectNew))
print(len(objectNew))

27102
27102


In [11]:
print(len(error))
error = list(set(error))
print(len(error))

789416
66487


In [12]:
for item in objectNew:
    src = vgRoot+'VG_100K/' + str(item) + '.jpg'
    dst = vgRoot+'Image100/' + str(item) + '.jpg'
    shutil.copy(src=src, dst=dst)

In [63]:
def get_bbox_attr(object_id, objectList):
    bboxObj = None
    nameObj = None
    attr = None
    status = True
    for item in objectList:
        if item['object_id'] == object_id:
            if(len(item['synsets']) == 0):
                status = False
                break
            else:
                bboxObj = [item['x'], item['y'], item['w'], item['h']]
                nameObj = item['synsets'][0].split('.')[0].replace('_','')
                if 'attributes' in item.keys():
                    attr = item['attributes']
                else:
                    attr = ['none', 'none']
                break
    return bboxObj, nameObj, attr, status

def get_attr_id(attr, att_cls_id):
    att_id = []
    for item in attr:
        att_id.append(att_cls_id.index(item))
    
    # if(len(att_id)>=2):
    #     att_id = att_id[:2]
    # else:
    #     att_id.append(att_cls_id.index('n/a'))
    return att_id

def get_attr_sysn(attr, att_sysn):
    att_id = []
    for item in attr:
        try:
            att_id.append(att_sysn[item])
        except:
            att_id.append(att_sysn["none"])
    
    if(len(att_id)>=2):
        att_id = att_id[:2]
    else:
        att_id.append(att_sysn["none"])
    return att_id

def valid_string(st):
    if(any(i.isdigit() for i in st)):
        return True
    for char in st:
        if not char.isalpha():  # Kiểm tra nếu có ký tự không phải là chữ cái
            return True 
    for item in st.split(" "):
        if(len(item) <= 2):
            return True
    if(len(st.split(" "))>=3):
        return True

#print(valid_string( 'no parking sign'))

In [81]:
checkRel = [
        "",
        "above",
        "across",
        "against",
        "along",
        "and",
        "at",
        "attached to",
        "behind",
        "belonging to",
        "between",
        "carrying",
        "covered in",
        "covering",
        "eating",
        "flying in",
        "for",
        "from",
        "growing on",
        "hanging from",
        "has",
        "holding",
        "in",
        "in front of",
        "laying on",
        "looking at",
        "lying on",
        "made of",
        "mounted on",
        "near",
        "of",
        "on",
        "on back of",
        "over",
        "painted on",
        "parked on",
        "part of",
        "playing",
        "riding",
        "says",
        "sitting on",
        "standing on",
        "to",
        "under",
        "using",
        "walking in",
        "walking on",
        "watching",
        "wearing",
        "wears",
        "with"
    ]
print(len(checkRel))

51


In [173]:
# Using readlines()
file1 = open(vgRoot + 'Annotation/' +'object_cls.txt', 'r')
Lines = file1.readlines()
obj_cls = []
# Strips the newline character
for line in Lines:
    obj_cls.append(line.replace('\n', ''))

len(obj_cls)

32

In [149]:
# Using readlines()
file1 = open(vgRoot + 'Annotation/' +'att_cls.txt', 'r')
Lines = file1.readlines()
att_cls = []
# Strips the newline character
for line in Lines:
    att_cls.append(line.replace('\n', ''))

len(att_cls)

5455

In [28]:
c = []
for item in dataSG[:]:
    if(len(item['relationships']) >= 5):
        c.append(item['image_id'])
print(len(c))
c = list(set(c))
print(len(c))

98629
98629


In [86]:
checkIsValid = []
checkObject = []
for item in dataSG[:]:
    #if((len(item['relationships']) >= 5)):
    objectGT = []
    objectObj = item['objects']
    for x in item['relationships']:
        bboxSub, nameSub, attrSub, statusSub = get_bbox_attr(x['subject_id'], objectObj)
        bboxObj, nameObj, attrObj, statusObj = get_bbox_attr(x['object_id'], objectObj)
        if(statusSub == True and statusObj == True):
            if((nameSub in objectNorm and nameObj in objectNorm) or (nameSub in objectNormTraffic and nameObj in objectNormTraffic)):
                if(x['predicate'].lower() in checkRel):
                    checkIsValid.append(item['image_id'])
                    gtDict = {
                            'rel': x['predicate'].lower(),

                            'bbox_sub': bboxSub,
                            'bbox_obj': bboxObj,

                            'attr_sub': get_attr_sysn(attrSub, att_sysn=attr_sysn),
                            'attr_obj': get_attr_sysn(attrObj, att_sysn=attr_sysn),

                            'cls_sub': nameSub,
                            'cls_obj': nameObj,

                            'org_id_sub': x['subject_id'],
                            'org_id_obj': x['object_id'],
                        }
                    objectGT.append(gtDict)
                
        # break
    checkObject.append(objectGT[:5])
    #break
print(len(checkIsValid))
checkIsValid = list(set(checkIsValid))
print(len(checkIsValid))

513672
77181


In [87]:
print(len(checkObject))
lenItem = []
objName = []
for item in checkObject[:]:
    lenItem.append(len(item))
    for x in item:
        objName.append(x['cls_sub'])
        objName.append(x['cls_obj'])

print(sum(lenItem) / len(lenItem) )
print(len(objName))
objName = list(set(objName))
print(len(objName))
D = {i+1: objName[i] for i in range(len(objName))}
D
with open(vgRoot + 'Annotation/' +'object_cls.json', "w") as outfile:
    json.dump(D, outfile)

108077
2.707190243992709
585170
178


In [88]:
attrName = []
for item in checkObject[:]:
    for x in item:
        listAtt = x['attr_obj'] + x['attr_sub']
        for at in listAtt:
            attrName.append(at)

print(len(attrName))
attrName = list(set(attrName))
print(len(attrName))

1170340
3488


In [90]:
relName = []
for item in checkObject[:]:
    for x in item:
        relName.append(x['rel'])

print(len(relName))
relName = list(set(relName))
print(len(relName))

292585
51


In [92]:
checkObject[-1]

[{'rel': 'on',
  'bbox_sub': [238, 233, 89, 54],
  'bbox_obj': [14, 110, 289, 193],
  'attr_sub': ['none.s.01', 'none.s.01'],
  'attr_obj': ['green.s.01', 'none.s.01'],
  'cls_sub': 'plate',
  'cls_obj': 'bus',
  'org_id_sub': 3786799,
  'org_id_obj': 3103225},
 {'rel': 'with',
  'bbox_sub': [14, 110, 289, 193],
  'bbox_obj': [98, 127, 50, 144],
  'attr_sub': ['green.s.01', 'none.s.01'],
  'attr_obj': ['none.s.01', 'none.s.01'],
  'cls_sub': 'bus',
  'cls_obj': 'door',
  'org_id_sub': 3103225,
  'org_id_obj': 2712437},
 {'rel': 'on',
  'bbox_sub': [100, 130, 50, 113],
  'bbox_obj': [29, 109, 225, 155],
  'attr_sub': ['none.s.01', 'none.s.01'],
  'attr_obj': ['green.s.01', 'none.s.01'],
  'cls_sub': 'door',
  'cls_obj': 'bus',
  'org_id_sub': 3561727,
  'org_id_obj': 3140197}]

In [93]:
# open file in write mode
with open(vgRoot + 'Annotation/' +'att_cls.txt', 'w') as fp:
    for item in attrName:
        # write each item on a new line
        fp.write("%s\n" % item.strip())
    print('Done')

Done


In [94]:
# open file in write mode
with open(vgRoot + 'Annotation/' +'object_cls.txt', 'w') as fp:
    for item in objName:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [95]:
# open file in write mode
with open(vgRoot + 'Annotation/' +'rel_cls.txt', 'w') as fp:
    for item in relName:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [292]:
# # Using readlines()
cls_obj = []
file1 = open(vgRoot+'Annotation/object_cls.txt', 'r')
Lines = file1.readlines()

count = 0
# Strips the newline character
for line in Lines:
    count += 1
    cls_obj.append(line.strip().lower().replace('\n',''))
print(len(cls_obj))

227


In [293]:
# # Using readlines()
cls_rel = []
file1 = open(vgRoot+'Annotation/rel_cls.txt', 'r')
Lines = file1.readlines()

count = 0
# Strips the newline character
for line in Lines:
    count += 1
    cls_rel.append(line.strip().lower().replace('\n',''))
print(len(cls_rel))

62


In [294]:
# # Using readlines()
cls_att = []
file1 = open(vgRoot+'Annotation/att_cls.txt', 'r')
Lines = file1.readlines()

count = 0
# Strips the newline character
for line in Lines:
    count += 1
    cls_att.append(line.strip().lower().replace('\n',''))
print(len(cls_att))

235


In [309]:
for file_name in os.listdir(vgRoot + 'GTTraffic'):
    print(file_name)
    dataId = []
    with open(vgRoot + 'GTTraffic/' + file_name, "r+") as jsonFile:
        data = json.load(jsonFile)
        for x in data:
            # x["rel_id"] = cls_rel.index(x['rel'])
            # x['id_sub'] = cls_obj.index(x['cls_sub'])
            # x['id_obj'] = cls_obj.index(x['cls_obj'])
            x['attr_sub_id'] = get_attr_id(x['attr_sub'], cls_att)
            x['attr_obj_id'] = get_attr_id(x['attr_obj'], cls_att)
            dataId.append(x)
        jsonFile.seek(0)
        json.dump(dataId, jsonFile)
        jsonFile.truncate()
    #break

2407777.json
2397743.json
2371916.json
2407707.json
2387695.json
2345106.json
2388704.json
2377672.json
2357303.json
2406609.json
2386873.json
2360329.json
2383914.json
2406268.json
2343888.json
2348659.json
2370884.json
2332584.json
2382640.json
2380434.json
2316656.json
2399833.json
2399772.json
2362848.json
2403943.json
2379557.json
2384444.json
2345410.json
2364869.json
2400467.json
2317713.json
2377209.json
2387515.json
2358707.json
2324588.json
2358500.json
2347620.json
2407554.json
2356824.json
2353122.json
2381896.json
2365540.json
2364705.json
2401317.json
2378295.json
2364821.json
2412494.json
2374506.json
2409934.json
2393143.json
2401761.json
2401315.json
2378323.json
2406102.json
2375953.json
2409925.json
2934.json
2385033.json
2401573.json
2326905.json
2381641.json
2384692.json
2392288.json
2388865.json
2403392.json
2408137.json
2360150.json
713550.json
2342053.json
2319417.json
2381943.json
2396921.json
2404213.json
2348070.json
2384398.json
2385397.json
2318714.json
235

In [312]:
for file_name in os.listdir(vgRoot + 'GTTraffic'):
    print(file_name)
    src = vgRoot+'VG_100K/' + file_name.replace('.json', '.jpg')
    dst = vgRoot+'ImageTraffic/' + file_name.replace('.json', '.jpg')
    shutil.copy(src=src, dst=dst)

2407777.json
2397743.json
2371916.json
2407707.json
2387695.json
2345106.json
2388704.json
2377672.json
2357303.json
2406609.json
2386873.json
2360329.json
2383914.json
2406268.json
2343888.json
2348659.json
2370884.json
2332584.json
2382640.json
2380434.json
2316656.json
2399833.json
2399772.json
2362848.json
2403943.json
2379557.json
2384444.json
2345410.json
2364869.json
2400467.json
2317713.json
2377209.json
2387515.json
2358707.json
2324588.json
2358500.json
2347620.json
2407554.json
2356824.json
2353122.json
2381896.json
2365540.json
2364705.json
2401317.json
2378295.json
2364821.json
2412494.json
2374506.json
2409934.json
2393143.json
2401761.json
2401315.json
2378323.json
2406102.json
2375953.json
2409925.json
2934.json
2385033.json
2401573.json
2326905.json
2381641.json
2384692.json
2392288.json
2388865.json
2403392.json
2408137.json
2360150.json
713550.json
2342053.json
2319417.json
2381943.json
2396921.json
2404213.json
2348070.json
2384398.json
2385397.json
2318714.json
235

In [3]:
targets = {"boxes": [[0.5680, 0.3720, 0.5196, 0.0860],
        [0.1979, 0.4735, 0.3656, 0.8890],
        [0.2553, 0.1660, 0.2266, 0.1660],
        [0.3512, 0.9135, 0.3218, 0.1270],
        [0.8301, 0.0765, 0.0801, 0.0430],
        [0.4887, 0.8690, 0.0770, 0.0380],
        [0.3618, 0.9505, 0.1767, 0.0610],
        [0.6405, 0.1740, 0.7190, 0.3240],
        [0.5008, 0.7420, 0.9985, 0.1200],
        [0.6745, 0.1575, 0.6511, 0.2690]], 
"labels": [103,  30,  44,  48,  99, 101, 101, 115, 124, 115], 
"image_id": [498335], 
"area":[ 43752.2266, 314573.4062,  36457.5156,  39950.8594,   3448.0918,2880.2224,  10529.6729, 225584.3281, 115954.8594, 169860.1250], 
"iscrowd":[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
"orig_size":[1000,  662], 
"size":[1208,  800], 
"rel_annotations":[[ 1,  2, 50],[ 3,  6, 22]]
}

In [ ]:
import torch
target_rels_boxes = torch.cat([t['boxes'][t["rel_annotations"][i, 0]] for t, (_, i) in zip(targets, indices[1])], dim=0)